<a href="https://colab.research.google.com/github/penny1xu/Capstone-Project/blob/main/Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install PyRadiomics if not installed already
!pip install pyradiomics

In [ ]:
# importing libraries
from __future__ import print_function
import sys
import os
import collections
import SimpleITK as sitk
from SimpleITK.SimpleITK import JoinSeries
import logging
import six
import radiomics
from radiomics import featureextractor, getFeatureClasses, getTestCase, firstorder, glcm, imageoperations, shape, glrlm, glszm
import numpy as np
import pandas as pd
import seaborn as sns
import zipfile
import PIL
from PIL import Image
from google.colab import files
import pandas as pd
import csv

In [ ]:
!wget --no-check-certificate \
    "https://github.com/lukevrobbins/covid_19_severity_prediction/archive/refs/heads/Justin.zip" \
    -O "/tmp/chest-x-rays.zip"

zip_ref = zipfile.ZipFile('/tmp/covid_19_severity_prediction-Justin/Chest X-ray Image Data sets.zip', 'r') # Opens the zip file in read mode
zip_ref.extractall('/tmp') # Extracts the files into the /tmp folder
zip_ref.close()


In [ ]:
# First define the settings of extractor
settings = {}
settings['label'] = 255
#settings['binWidth'] = 25
#settings['resampledPixelSpacing'] = None
#settings['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
#settings['interpolator'] = 'sitkBSpline'
#settings['verbose'] = True
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
print('Extraction parameters:\n\t', extractor.settings)

# Enable a filter (in addition to the 'Original' filter already enabled)
extractor.enableImageTypeByName('LoG')
print('')
print('Enabled filters:\n\t', extractor.enabledImagetypes)

# Specify feature classes, all are enabled by default
extractor.disableAllFeatures()
extractor.enableAllFeatures()
#extractor.enableFeatureClassByName('firstorder')

# Specify some additional features in the GLCM feature class
#extractor.enableFeaturesByName(glcm=['Autocorrelation', 'Homogeneity1', 'SumSquares'])
print('')
print('Enabled features:\n\t', extractor.enabledFeatures)


In [ ]:
dataDir = os.path.join('/tmp/Chest X-ray Image Data sets')
print("dataDir, relative path:", dataDir)
print("dataDir, absolute path:", os.path.abspath(dataDir))

count = 1
field_value= []
field_names= []
for file in os.listdir(dataDir):
  if(count > len(os.listdir(dataDir))/2):
    break
  # Define the testcase name
  CXR_name = str(count)
  # Store the file paths of our testing image and label map into two variables
  imagePath = os.path.join(dataDir, CXR_name + ".jpg")
  maskPath = os.path.join(dataDir, CXR_name + "-mask.png")
  # Transforming into "3D" image for pyradiomics
  image = sitk.ReadImage(imagePath, sitk.sitkInt16)
  image3d = sitk.JoinSeries(image)
  mask = sitk.ReadImage(maskPath, sitk.sitkInt16)
  mask3d = sitk.JoinSeries(mask)

  # Extract features
  result = extractor.execute(image3d, mask3d)
  # print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
  # print('Features extracted for image ' + str(count))
  # print('Result type:', type(result))  # result is returned in a Python ordered dictionary
  # print('')
  # print('Calculated features')
  for key, value in six.iteritems(result):
      field_names.append(key)
      field_value.append(value)
  with open("frequencies.csv", "w") as outfile:
      csvwriter = csv.writer(outfile)
      csvwriter.writerow(field_names)
      csvwriter.writerow(field_value)
  field_value= []
  field_names= []
  count = count + 1


    

In [ ]:
CXR_name = "1"

imagePath = os.path.join(dataDir, CXR_name + ".jpg")
maskPath = os.path.join(dataDir, CXR_name + "-mask.png")

image = sitk.ReadImage(imagePath, sitk.sitkInt16)
image3d = sitk.JoinSeries(image)
mask = sitk.ReadImage(maskPath, sitk.sitkInt16)
mask3d = sitk.JoinSeries(mask)
result = extractor.execute(image3d, mask3d)

for key, value in six.iteritems(result):
    field_names.append(key)
    field_value.append(value)
with open("frequencies.csv", "w") as outfile:
    csvwriter = csv.writer(outfile)
    csvwriter.writerow(field_names)
    csvwriter.writerow(field_value)

parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Image too small to apply LoG filter, size: [256 256   1]
